#### Cosine Similarity (BERT version)

Cosine similarity is used to find the angle between vectors. As the embeddings are vectors, it is used to find the similarity between games. Each game will be compared to all games creating a matrix of n games x n games.

I created a new notebook as a continuation as the previous one was getting buggy. Let's try by reading the pickle files and csv

In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import gc

In [3]:
tfidf_matrix = pd.read_pickle("TF-IDF_V1.pkl")
embeddings_df = pd.read_pickle("embeddings_V1.pkl")
sg_df_clean = pd.read_csv("sg_df_clean.csv")

In [4]:
embedding_matrix = np.array(embeddings_df.to_list()) #we need to "unpack" the column to create a matrix to use for cosine similarity

Let's run the cosine similarity for both TF-IDF and BERT embeddings

In [5]:
sim_matrix_tfidf = cosine_similarity(tfidf_matrix)
sim_matrix_embeddings = cosine_similarity(embedding_matrix)

In [6]:
sim_matrix_embeddings.shape

(72371, 72371)

In [7]:
sim_matrix_tfidf.shape

(72371, 72371)

Due to to the sheer size of the matrix, I need to convert the items from fp64 to fp32 (reduce memory usage) and delete unnecessary variables

In [9]:
del tfidf_matrix
del embeddings_df
del embedding_matrix

In [10]:
sim_matrix_tfidf = sim_matrix_tfidf.astype(np.float32) #changing format from float64 to float32 to save memory

In [11]:
gc.collect() #This code is used for clearing some memory

0

In [12]:
sim_matrix_embeddings = sim_matrix_embeddings.astype(np.float32)

In [13]:
gc.collect()

0

I am going to assign "weights" for each similarity matrix. For the first "recommendations engine" I am going to assign equal weights to tf-idf and BERT. I will change this later on for other models depending on the results.

In [14]:
tfidf_w = 0.5
embeddings_w = 0.5

In [16]:
weighted_matrix_1 = tfidf_w * sim_matrix_tfidf #multiplying by the weight

In [17]:
gc.collect()

0

In [18]:
del sim_matrix_tfidf #to clear memory

In [19]:
weighted_matrix_2 =embeddings_w * sim_matrix_embeddings #multiplying by the weight

In [20]:
gc.collect()

0

In [21]:
del sim_matrix_embeddings #clearing memory

In [43]:
final_matrix = weighted_matrix_1 + weighted_matrix_2 #summing the matrices to get the final matrix

In [45]:
gc.collect()

1032

In [47]:
del weighted_matrix_1 #clearing memory
del weighted_matrix_2 

I am going to export the final matrix into a pickle file and create a new notebook to reduce the likelihood of memory issues

In [49]:
pd.to_pickle(final_matrix, "Full_cosine_matrix.pkl")